In [1]:
# =============================================================================
# Tools - Giving LLMs the Ability to Take Actions (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
Tools - Extending LLM Capabilities
=====================================================================

Tools allow LLMs to interact with external systems, APIs, and data.
They transform LLMs from text generators into action-taking agents.

Key Concepts:
-------------
1. @tool decorator - Simplest way to create tools
2. bind_tools() - Attach tools to a model
3. Tool execution - Handle tool calls and responses
4. Tool schemas - Automatic schema generation from type hints

Tool Calling Flow:
------------------
    User Question
         │
         ▼
    ┌─────────────────┐
    │  LLM with Tools │
    │  (bind_tools)   │
    └─────────────────┘
         │
         ▼ (LLM decides to use tool)
    ┌─────────────────┐
    │   Tool Call     │  {"name": "get_weather", "args": {"location": "NYC"}}
    │   (AIMessage)   │
    └─────────────────┘
         │
         ▼ (Execute tool)
    ┌─────────────────┐
    │  Tool Result    │  "The weather in NYC is sunny"
    │  (ToolMessage)  │
    └─────────────────┘
         │
         ▼
    ┌─────────────────┐
    │  Final Answer   │  LLM uses tool result to respond
    └─────────────────┘

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ Environment configured for tools examples")

✅ Environment configured for tools examples


In [2]:
# =============================================================================
# Tool Creation and Binding (LangChain 1.0+)
# =============================================================================
"""
Three Ways to Create Tools:
---------------------------
1. @tool decorator - Simplest, uses function signature
2. StructuredTool - More control over schema
3. BaseTool class - Full customization

bind_tools() - The Key Method:
------------------------------
Attaches tools to a model, enabling the LLM to:
- See available tools and their schemas
- Decide when to call tools
- Generate proper tool call arguments
"""

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from pydantic import BaseModel, Field
from typing import Optional

# =============================================================================
# Method 1: @tool Decorator (Recommended for Most Cases)
# =============================================================================
"""
The @tool decorator automatically:
- Extracts the function name as tool name
- Uses the docstring as tool description
- Generates schema from type hints
"""

@tool
def get_weather(location: str) -> str:
    """
    Get the current weather for a given location.
    
    Args:
        location: The city or location to get weather for (e.g., "New York", "London")
    
    Returns:
        A string describing the current weather conditions
    """
    # In production, this would call a real weather API
    return f"The weather in {location} is sunny with a high of 75°F."

@tool
def search_database(query: str, limit: int = 5) -> str:
    """
    Search the database for relevant information.
    
    Args:
        query: The search query
        limit: Maximum number of results to return (default: 5)
    
    Returns:
        Search results as a formatted string
    """
    return f"Found {limit} results for '{query}': [Result 1, Result 2, ...]"

# Check the tool schema
print("=" * 60)
print("Tool Schema (auto-generated from type hints):")
print("=" * 60)
print(f"Name: {get_weather.name}")
print(f"Description: {get_weather.description}")
print(f"Args Schema: {get_weather.args_schema.schema() if get_weather.args_schema else 'None'}")

# =============================================================================
# Method 2: Using Pydantic for Complex Schemas
# =============================================================================
"""
For tools with complex input validation, use a Pydantic model
with the args_schema parameter.
"""

class WeatherInput(BaseModel):
    """Input schema for weather tool with validation."""
    location: str = Field(..., description="City name or location", min_length=2)
    units: Optional[str] = Field(default="fahrenheit", description="Temperature units: 'fahrenheit' or 'celsius'")

@tool(args_schema=WeatherInput)
def get_weather_detailed(location: str, units: str = "fahrenheit") -> str:
    """Get detailed weather with unit selection."""
    temp = 75 if units == "fahrenheit" else 24
    return f"Weather in {location}: {temp}°{'F' if units == 'fahrenheit' else 'C'}, sunny"

print("\n" + "=" * 60)
print("Tool with Pydantic Schema:")
print("=" * 60)
print(f"Schema: {get_weather_detailed.args_schema.schema()}")

/Users/naveen/Desktop/web/genAi/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tool Schema (auto-generated from type hints):
Name: get_weather
Description: Get the current weather for a given location.

Args:
    location: The city or location to get weather for (e.g., "New York", "London")

Returns:
    A string describing the current weather conditions
Args Schema: {'description': 'Get the current weather for a given location.\n\nArgs:\n    location: The city or location to get weather for (e.g., "New York", "London")\n\nReturns:\n    A string describing the current weather conditions', 'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weather', 'type': 'object'}

Tool with Pydantic Schema:
Schema: {'description': 'Input schema for weather tool with validation.', 'properties': {'location': {'description': 'City name or location', 'minLength': 2, 'title': 'Location', 'type': 'string'}, 'units': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': 'fahrenheit', 'description': "Temperature units: '

/var/folders/nb/pbswtk4n7px0p0yxxc0z0lzh0000gn/T/ipykernel_6269/2512067607.py:69: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"Args Schema: {get_weather.args_schema.schema() if get_weather.args_schema else 'None'}")
/var/folders/nb/pbswtk4n7px0p0yxxc0z0lzh0000gn/T/ipykernel_6269/2512067607.py:93: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"Schema: {get_weather_detailed.args_schema.schema()}")


In [3]:
# =============================================================================
# Binding Tools to Models (LangChain 1.0+)
# =============================================================================
"""
bind_tools() - Attaching Tools to LLMs
--------------------------------------
The bind_tools() method is how you give an LLM access to tools.
It returns a new model instance with tools attached.

Supported Providers (2025):
- OpenAI (gpt-4o, gpt-4o-mini, gpt-4-turbo)
- Anthropic (claude-3.5-sonnet, claude-3-opus)
- Google (gemini-1.5-pro, gemini-1.5-flash)
- Groq (llama-3.1-70b, mixtral-8x7b)
- Mistral, Cohere, and more

Important: bind_tools() creates a NEW model instance
           The original model is unchanged
"""

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# Initialize model
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)

# Define tools
tools = [get_weather, search_database, get_weather_detailed]

# Bind tools to model - creates a new model with tool access
model_with_tools = model.bind_tools(tools)

print("=" * 60)
print("Model with Tools Bound")
print("=" * 60)
print(f"Original model: {type(model).__name__}")
print(f"Model with tools: {type(model_with_tools).__name__}")
print(f"Tools available: {[t.name for t in tools]}")

# =============================================================================
# How Tool Calling Works
# =============================================================================
"""
When you invoke a model with bound tools:

1. LLM sees the user's question + available tool schemas
2. LLM decides IF it needs to use a tool
3. If yes: Returns AIMessage with tool_calls populated
4. If no: Returns regular AIMessage with content

The tool_calls are structured as:
{
    "name": "tool_name",
    "args": {"arg1": "value1", "arg2": "value2"},
    "id": "unique_call_id"
}
"""

# Example: Ask something that requires a tool
response = model_with_tools.invoke([
    HumanMessage(content="What's the weather in San Francisco?")
])

print("\n" + "=" * 60)
print("Tool Call Response Analysis")
print("=" * 60)
print(f"Response type: {type(response).__name__}")
print(f"Has tool calls: {bool(response.tool_calls)}")
if response.tool_calls:
    print(f"\nTool calls made:")
    for tc in response.tool_calls:
        print(f"  - Tool: {tc['name']}")
        print(f"    Args: {tc['args']}")
        print(f"    ID: {tc['id']}")

Model with Tools Bound
Original model: ChatOpenAI
Model with tools: RunnableBinding
Tools available: ['get_weather', 'search_database', 'get_weather_detailed']

Tool Call Response Analysis
Response type: AIMessage
Has tool calls: True

Tool calls made:
  - Tool: get_weather
    Args: {'location': 'San Francisco'}
    ID: call_bWqGG0Y1W3iS1MpOb0pyILNq


In [4]:
# =============================================================================
# Executing Tools and Getting Final Answers
# =============================================================================
"""
Complete Tool Execution Flow:
-----------------------------
1. User asks a question
2. Model returns AIMessage with tool_calls
3. YOU execute the tool with the provided args
4. Send ToolMessage with the result back to the model
5. Model generates final answer using the tool result

Message Flow:
-------------
    [HumanMessage]     "What's the weather in NYC?"
           │
           ▼
    [AIMessage]        tool_calls: [{name: "get_weather", args: {location: "NYC"}}]
           │
           ▼ (You execute the tool)
    [ToolMessage]      "The weather in NYC is sunny, 75°F"
           │
           ▼
    [AIMessage]        "The weather in New York City is currently sunny..."
"""

def execute_tools(tool_calls: list, available_tools: dict) -> list:
    """
    Execute tool calls and return ToolMessages with results.
    
    Args:
        tool_calls: List of tool calls from AIMessage
        available_tools: Dict mapping tool names to tool functions
    
    Returns:
        List of ToolMessage objects with results
    """
    tool_messages = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        # Find and execute the tool
        if tool_name in available_tools:
            result = available_tools[tool_name].invoke(tool_args)
        else:
            result = f"Error: Tool '{tool_name}' not found"
        
        # Create ToolMessage with the result
        tool_messages.append(ToolMessage(
            content=str(result),
            tool_call_id=tool_id  # Must match the original tool call ID
        ))
    
    return tool_messages

# Create tool lookup dictionary
tools_by_name = {tool.name: tool for tool in tools}

# Complete conversation with tool execution
messages = [HumanMessage(content="What's the weather in Tokyo?")]

print("=" * 60)
print("Complete Tool Execution Flow")
print("=" * 60)

# Step 1: Get tool call from model
print("\n📝 Step 1: User asks question")
print(f"   Message: {messages[0].content}")

response = model_with_tools.invoke(messages)
messages.append(response)

# Step 2: Execute tools
print("\n🔧 Step 2: Model decides to use tool")
if response.tool_calls:
    for tc in response.tool_calls:
        print(f"   Tool: {tc['name']}")
        print(f"   Args: {tc['args']}")
    
    # Execute and add results
    tool_results = execute_tools(response.tool_calls, tools_by_name)
    messages.extend(tool_results)
    
    print("\n📊 Step 3: Tool execution result")
    for tm in tool_results:
        print(f"   Result: {tm.content}")
    
    # Step 4: Get final answer
    final_response = model_with_tools.invoke(messages)
    
    print("\n✅ Step 4: Model generates final answer")
    print(f"   Answer: {final_response.content}")
else:
    print("   No tools needed for this question")
    print(f"   Direct answer: {response.content}")

Complete Tool Execution Flow

📝 Step 1: User asks question
   Message: What's the weather in Tokyo?

🔧 Step 2: Model decides to use tool
   Tool: get_weather
   Args: {'location': 'Tokyo'}

📊 Step 3: Tool execution result
   Result: The weather in Tokyo is sunny with a high of 75°F.

✅ Step 4: Model generates final answer
   Answer: The weather in Tokyo is sunny with a high of 75°F.


In [5]:
# =============================================================================
# Agent Loop with Tools (LangChain 1.0+ Pattern)
# =============================================================================
"""
Building a Simple Agent Loop
----------------------------
An agent is just a loop that:
1. Takes user input
2. Calls LLM with tools
3. Executes any tool calls
4. Repeats until no more tool calls
5. Returns final answer

This is the foundation of how agents work in LangChain/LangGraph.
"""

def run_agent(user_input: str, max_iterations: int = 5) -> str:
    """
    Simple agent loop that handles tool calling.
    
    Args:
        user_input: The user's question
        max_iterations: Maximum tool execution cycles (safety limit)
    
    Returns:
        The agent's final response
    """
    messages = [HumanMessage(content=user_input)]
    
    for i in range(max_iterations):
        # Get model response
        response = model_with_tools.invoke(messages)
        messages.append(response)
        
        # Check if model wants to use tools
        if not response.tool_calls:
            # No tool calls = we have our final answer
            return response.content
        
        # Execute all tool calls
        print(f"  🔧 Iteration {i+1}: Executing {len(response.tool_calls)} tool(s)")
        for tc in response.tool_calls:
            print(f"     - {tc['name']}({tc['args']})")
        
        tool_results = execute_tools(response.tool_calls, tools_by_name)
        messages.extend(tool_results)
    
    # Safety: return last response if we hit max iterations
    return messages[-1].content if messages else "Max iterations reached"

# Test the agent
print("=" * 60)
print("Agent Loop Example")
print("=" * 60)

questions = [
    "What's the weather in Paris?",
    "Search for information about Python programming",
]

for q in questions:
    print(f"\n📝 Question: {q}")
    print("-" * 40)
    answer = run_agent(q)
    print(f"✅ Answer: {answer[:200]}..." if len(answer) > 200 else f"✅ Answer: {answer}")

Agent Loop Example

📝 Question: What's the weather in Paris?
----------------------------------------
  🔧 Iteration 1: Executing 1 tool(s)
     - get_weather({'location': 'Paris'})
✅ Answer: The weather in Paris is sunny with a high of 75°F.

📝 Question: Search for information about Python programming
----------------------------------------
  🔧 Iteration 1: Executing 1 tool(s)
     - search_database({'query': 'Python programming'})
✅ Answer: I found some information about Python programming. Here are the results:

1. **Result 1**: Overview of Python programming language, its features, and applications.
2. **Result 2**: A guide on how to g...


In [6]:
# =============================================================================
# Advanced: Tool Choice and Forcing Tool Usage
# =============================================================================
"""
Controlling Tool Selection
--------------------------
bind_tools() accepts a tool_choice parameter to control how the model
uses tools:

Options:
- "auto" (default): Model decides when to use tools
- "any" / "required": Model MUST use at least one tool
- "none": Model cannot use tools (even if bound)
- {"type": "tool", "name": "tool_name"}: Force specific tool

Use Cases:
- tool_choice="required": Ensure tool is always called
- tool_choice={"type": "tool", "name": "get_weather"}: Force weather lookup
- tool_choice="none": Temporarily disable tools
"""

# Force model to use a specific tool
model_forced_weather = model.bind_tools(
    tools,
    tool_choice={"type": "function", "function": {"name": "get_weather"}}
)

# Force model to use ANY tool (but must use one)
model_must_use_tool = model.bind_tools(
    tools,
    tool_choice="required"
)

print("=" * 60)
print("Tool Choice Examples")
print("=" * 60)

# Example 1: Forced to use get_weather
print("\n📌 Forced to use 'get_weather' tool:")
response = model_forced_weather.invoke([
    HumanMessage(content="Tell me about machine learning")  # Unrelated question!
])
if response.tool_calls:
    print(f"   Tool called: {response.tool_calls[0]['name']}")
    print(f"   Args: {response.tool_calls[0]['args']}")
else:
    print(f"   Direct response: {response.content[:100]}...")

# Example 2: Must use some tool
print("\n📌 Required to use a tool (any tool):")
response = model_must_use_tool.invoke([
    HumanMessage(content="What's happening in London today?")
])
if response.tool_calls:
    print(f"   Tool called: {response.tool_calls[0]['name']}")
else:
    print("   No tool called (unexpected)")

Tool Choice Examples

📌 Forced to use 'get_weather' tool:
   Tool called: get_weather
   Args: {'location': 'machine learning'}

📌 Required to use a tool (any tool):
   Tool called: get_weather


In [7]:
# =============================================================================
# Summary: Tools in LangChain 1.0+
# =============================================================================
"""
=====================================================================
KEY TAKEAWAYS - Tools in LangChain
=====================================================================

1. CREATING TOOLS:
   ----------------
   @tool
   def my_tool(arg: str) -> str:
       '''Docstring becomes tool description'''
       return result

   # With Pydantic validation:
   @tool(args_schema=MyInputSchema)
   def validated_tool(arg: str) -> str: ...

2. BINDING TOOLS:
   ---------------
   model_with_tools = model.bind_tools([tool1, tool2])
   
   # With tool choice:
   model.bind_tools(tools, tool_choice="required")
   model.bind_tools(tools, tool_choice={"type": "function", "function": {"name": "my_tool"}})

3. TOOL EXECUTION FLOW:
   ---------------------
   response = model_with_tools.invoke(messages)
   
   if response.tool_calls:
       for tc in response.tool_calls:
           result = tools_dict[tc["name"]].invoke(tc["args"])
           messages.append(ToolMessage(content=result, tool_call_id=tc["id"]))
       
       final = model_with_tools.invoke(messages)

4. AGENT LOOP PATTERN:
   --------------------
   while True:
       response = model_with_tools.invoke(messages)
       if not response.tool_calls:
           break
       # Execute tools, add results to messages

5. BEST PRACTICES:
   ----------------
   - Write clear docstrings (they become tool descriptions)
   - Use type hints (they define the schema)
   - Use Pydantic for complex validation
   - Always include tool_call_id in ToolMessage
   - Set max_iterations to prevent infinite loops
   - Use temperature=0 for consistent tool calling

Common Imports:
---------------
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

=====================================================================
"""

print("=" * 60)
print("Tools Module Complete!")
print("=" * 60)
print("""
Next Steps:
-----------
1. 4-messages.ipynb - Message types and conversation management
2. 5-structureoutput.ipynb - Getting structured data from LLMs
3. 6-middleware.ipynb - Request/response middleware

For more complex agent patterns, see:
- LangGraph for stateful, multi-step agents
- projects2/5agent-guardrails.py for agent with constraints
""")

Tools Module Complete!

Next Steps:
-----------
1. 4-messages.ipynb - Message types and conversation management
2. 5-structureoutput.ipynb - Getting structured data from LLMs
3. 6-middleware.ipynb - Request/response middleware

For more complex agent patterns, see:
- LangGraph for stateful, multi-step agents
- projects2/5agent-guardrails.py for agent with constraints

